In [1]:
import pandas as pd
import os
from langchain.chat_models import ChatOpenAI
from langchain.utilities import PythonREPL
from langchain.agents import Tool, create_pandas_dataframe_agent, initialize_agent
from langchain.memory import ConversationBufferMemory
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

def get_RSI(price, lookback=14):
    change = price.copy().diff()
    change.dropna(inplace=True)

    change_up = change.copy()
    change_down = change.copy()

    change_up[change_up<0] = 0
    change_down[change_down>0] = 0

    # Calculate the rolling average of average up and average down
    avg_up = change_up.rolling(lookback).mean()
    avg_down = change_down.rolling(lookback).mean().abs()
    rsi = 100 * avg_up / (avg_up + avg_down)
    return rsi

def get_sma(price, lookback):
    """referenced from vectorize me slides"""
    sma = price.rolling(window=lookback, min_periods=lookback).mean()
    return sma

def get_golden_or_death_cross(price, golden, return_sma=False):
    short_lookback = 10
    long_lookback = 30
    sma_short = get_sma(price, short_lookback)
    sma_long = get_sma(price, long_lookback)
    sma = sma_short.join(sma_long, lsuffix=f'_SMA_{short_lookback}', rsuffix=f'_SMA_{long_lookback}')
    sma.rename(columns={
        f'Close_SMA_{short_lookback}':f'SMA_{short_lookback}',
        f'Close_SMA_{long_lookback}':f'SMA_{long_lookback}',
    }, inplace=True)
    
    if golden:
        crossed = sma[f'SMA_{short_lookback}'] > sma[f'SMA_{long_lookback}']
    else:
        crossed = sma[f'SMA_{short_lookback}'] < sma[f'SMA_{long_lookback}']

    to_modify=[]
    for i in range(1, len(crossed)):
        if crossed.iloc[i-1] and crossed.iloc[i]:
            to_modify.append(i)
    crossed.iloc[to_modify] = False

    if return_sma:
        return crossed, sma
    return crossed

def get_data():
    sp500 = pd.read_csv('../data/sp500.csv', index_col='Date')
    spy = pd.read_csv('../data/sp500_index.csv', index_col='Date')
    spy['Ticker'] = 'SPY'
    spy.columns=['Close','Ticker']

    min_date, max_date = max(spy.index.min(), sp500.index.min()), min(spy.index.max(), sp500.index.max())
    dates = pd.DataFrame(index=pd.date_range(min_date, max_date))
    dates.index = dates.index.rename('Date').astype(str)

    sp500 = dates.join(sp500, how='left').fillna(method="ffill").fillna(method="bfill").sort_index(axis=0)
    spy = dates.join(spy, how='left').fillna(method="ffill").fillna(method="bfill").sort_index(axis=0)
    df = pd.concat([sp500, spy])

    new_dfs = []
    for idx, symbol in enumerate(df.Ticker.unique()):
        if idx % 10 == 0:
            print(f"{idx+1} / {len(df.Ticker.unique())} tickers")
        symbol_df = df[df.Ticker==symbol].reset_index()
        symbol_df.loc[:,'rsi'] = get_RSI(symbol_df[['Close']], 14)
        golden_cross,sma = get_golden_or_death_cross(symbol_df[['Close']], golden=True, return_sma=True)
        death_cross = get_golden_or_death_cross(symbol_df[['Close']], golden=False, return_sma=False)
        symbol_df.loc[:,'golden_cross'] = [1 if i else 0 for i in golden_cross]
        symbol_df.loc[:,'death_cross'] = [1 if i else 0 for i in death_cross] 
        symbol_df = symbol_df.join(sma, how='left')
        symbol_df.set_index('Date', inplace=True)
        new_dfs.append(symbol_df)

    df=pd.concat(new_dfs).reset_index()
    return df

In [2]:
llm = ChatOpenAI(model_name='gpt-4', temperature=0)
python_repl = PythonREPL()
agent = create_pandas_dataframe_agent(llm, get_data(), verbose=True)
tools = [
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command."
    ),
    Tool(
        name="Stock data",
        func=agent.run,
        description="useful for anything related to stock information"
    )
]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm_chain = initialize_agent(tools=tools, llm=llm, agent="chat-conversational-react-description", memory=memory, verbose=True)

1 / 322 tickers
11 / 322 tickers
21 / 322 tickers
31 / 322 tickers
41 / 322 tickers
51 / 322 tickers
61 / 322 tickers
71 / 322 tickers
81 / 322 tickers
91 / 322 tickers
101 / 322 tickers
111 / 322 tickers
121 / 322 tickers
131 / 322 tickers
141 / 322 tickers
151 / 322 tickers
161 / 322 tickers
171 / 322 tickers
181 / 322 tickers
191 / 322 tickers
201 / 322 tickers
211 / 322 tickers
221 / 322 tickers
231 / 322 tickers
241 / 322 tickers
251 / 322 tickers
261 / 322 tickers
271 / 322 tickers
281 / 322 tickers
291 / 322 tickers
301 / 322 tickers
311 / 322 tickers
321 / 322 tickers


In [4]:
# what is the rsi of netflix as of 2020-09-21?
# if i bought $1000 worth of Netflix at beginning of 2016, how much would i have made by the end of the year?
print("bot: Hi, I am Wealthsimple's trusted AI agent! How can I help?")
while True:
    res = llm_chain.run(input=input("user: "))
    print(f"bot: {res}\n")

bot: Hi, I am Wealthsimple's trusted AI agent! How can I help?


> Entering new AgentExecutor chain...
```json
{
    "action": "Stock data",
    "action_input": "get_rsi('NFLX', '2020-09-21')"
}
```

> Entering new AgentExecutor chain...
Thought: To get the RSI value for NFLX on 2020-09-21, I need to filter the dataframe for the given Ticker and Date, and then extract the 'rsi' column value.
Action: python_repl_ast
Action Input: df.loc[(df['Ticker'] == 'NFLX') & (df['Date'] == '2020-09-21'), 'rsi']
Observation: 521301    36.418701
Name: rsi, dtype: float64
Thought:I now know the RSI value for NFLX on 2020-09-21.
Final Answer: 36.418701

> Finished chain.

Observation: 36.418701
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The RSI of Netflix as of 2020-09-21 was 36.42."
}
```

> Finished chain.
bot: The RSI of Netflix as of 2020-09-21 was 36.42.



> Entering new AgentExecutor chain...


KeyboardInterrupt: 